In [1]:
import numpy as np

In [2]:
def adjust_depth_to_mean_line(depths, fov=80, threshold=45):
    #print("Starting adjust_depth_to_mean_line...")
    num_rays = len(depths)
    angles = np.radians(np.linspace(-fov / 2, fov / 2, num_rays))

    x_points = depths * np.sin(angles)
    y_points = depths * np.cos(angles)
    #print(f"x_points: {x_points}")
    #print(f"y_points: {y_points}")

    # Compute angles between consecutive points
    edge_angles = []
    for i in range(num_rays - 1):
        vec = np.array([x_points[i+1]-x_points[i], y_points[i+1]-y_points[i]])
        angle = np.degrees(np.arctan2(vec[1], vec[0]))
        edge_angles.append(angle)
    edge_angles = np.array(edge_angles)
    #print(f"edge_angles: {edge_angles}")

    # Identify break edges
    groups_of_points = []
    current_start = 0
    current_mean_angle = edge_angles[0] if len(edge_angles) > 0 else 0

    for i in range(1, len(edge_angles)):
        angle_jump = abs(edge_angles[i] - current_mean_angle)
        # If angle jump exceeds threshold, we end the current group at point i+1
        if angle_jump > threshold:
            # Group is from current_start to i+1 (points)
            groups_of_points.append((current_start, i))
            # Start new group from i+1
            current_start = i+1
            # Recompute mean angle for next group if possible
            if i < len(edge_angles) - 1:
                current_mean_angle = edge_angles[i+1]
            else:
                current_mean_angle = 0
        else:
            # Extend current group
            current_mean_angle = np.mean(edge_angles[current_start:i+1])

    # Add the last group
    groups_of_points.append((current_start, num_rays-1))  # ends at the last point index

    #print(f"Formed groups (in terms of points): {groups_of_points}")

    adjusted_depth = depths.copy()

    # Now fit a line to each group of points and project them
    for g_idx, (start_idx, end_idx) in enumerate(groups_of_points):
        # Points in this group are from start_idx to end_idx inclusive
        points_in_group = np.arange(start_idx, end_idx+1)  # +1 because end_idx is included
        x_group = x_points[points_in_group]
        y_group = y_points[points_in_group]

        if len(points_in_group) <= 1:
            continue

        slope, intercept = np.polyfit(x_group, y_group, 1)

        # Project all points in the group onto the best-fit line
        for i, point_idx in enumerate(points_in_group):
            x = x_points[point_idx]
            y = y_points[point_idx]
            x_proj = (x + slope*(y - intercept)) / (1 + slope**2)
            y_proj = slope*x_proj + intercept
            adjusted_depth[point_idx] = np.sqrt(x_proj**2 + y_proj**2)

    return adjusted_depth


In [13]:
import argparse
import os
import numpy as np
import torch
import tqdm
import yaml
from attrdict import AttrDict
import matplotlib.pyplot as plt

# Import your models
from modules.mono.depth_net_pl import depth_net_pl

# Import your dataset
from data_utils.data_utils import GridSeqDataset


import numpy as np

def plot_depth_rays(gt_depth, pred_depth, adjusted_depth, fov, camera_idx):
    """
    Plot GT, predicted, and adjusted depth rays for a given camera.
    
    Parameters:
    - gt_depth: Ground truth depth values.
    - pred_depth: Predicted depth values.
    - adjusted_depth: Depth values after adjustment.
    - fov: Field of view in degrees.
    - camera_idx: Camera index.
    """
    num_rays = len(gt_depth)
    angles = np.radians(np.linspace(-fov / 2, fov / 2, num_rays))

    # GT coordinates
    x_gt = gt_depth * np.sin(angles)
    y_gt = gt_depth * np.cos(angles)

    # Predicted coordinates
    x_pred = pred_depth * np.sin(angles)
    y_pred = pred_depth * np.cos(angles)

    # Adjusted coordinates
    x_adjusted = adjusted_depth * np.sin(angles)
    y_adjusted = adjusted_depth * np.cos(angles)
    
    # Compute MSE in depth domain (comparing depth values directly)
    mse_pred = np.mean((pred_depth - gt_depth) ** 2)
    mse_adj = np.mean((adjusted_depth - gt_depth) ** 2)
    
    # Plot GT, predicted, and adjusted rays
    plt.figure(figsize=(10, 10))
    plt.plot(x_gt, y_gt, label='GT Depth', marker='o')
    plt.plot(x_pred, y_pred, label='Predicted Depth', marker='x')
    plt.plot(x_adjusted, y_adjusted, label='Adjusted Depth', marker='s')
    plt.title(f'Top-Down View of Depth Rays - Camera {camera_idx}\nMSE (Pred): {mse_pred:.4f}, MSE (Adjusted): {mse_adj:.4f}')
    plt.xlabel('X-axis (meters)')
    plt.ylabel('Y-axis (meters)')
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()



def evaluate_depth_model(depth_net, test_set, device, fov):
    """
    Evaluate the depth model and plot GT and predicted depths for each camera.
    
    Parameters:
    - depth_net: Pre-trained depth network.
    - test_set: Dataset containing the camera data.
    - device: Device to run the model (CPU or GPU).
    - fov: Field of view in degrees.
    """
    mse_summary = {"camera_idx": [], "mse_pred": [], "mse_adjusted": []}

    for camera_idx, data in tqdm.tqdm(enumerate(test_set), total=len(test_set)):
        # if camera_idx not in [6]:
        #     continue
        # Ground truth depth
        gt_depth = data["ref_depth"]
        if len(gt_depth)>40:
            print(data['scene_name'])
            continue
        

        # Predict depth
        ref_img_torch = torch.tensor(data["ref_img"], device=device).unsqueeze(0)
        with torch.no_grad():
            pred_depths, _, _ = depth_net.encoder(ref_img_torch, None)
        pred_depth = pred_depths.squeeze(0).detach().cpu().numpy()
        
        # Adjust depth
        adjust_depth = adjust_depth_to_mean_line(pred_depth)

        # Compute MSE

        
        mse_pred = np.mean((pred_depth - gt_depth) ** 2)
        mse_adjusted = np.mean((adjust_depth - gt_depth) ** 2)

        # Store results for summary
        mse_summary["camera_idx"].append(camera_idx)
        mse_summary["mse_pred"].append(mse_pred)
        mse_summary["mse_adjusted"].append(mse_adjusted)

        # Plot GT, predicted, and adjusted depth rays
        # plot_depth_rays(gt_depth, pred_depth, adjust_depth, fov, camera_idx)

    # Print summary of MSEs
    # print("\nMSE Summary:")
    # for cam_idx, mse_p, mse_a in zip(mse_summary["camera_idx"], mse_summary["mse_pred"], mse_summary["mse_adjusted"]):
    #     print(f"Camera {cam_idx}: MSE (Pred): {mse_p:.4f}, MSE (Adjusted): {mse_a:.4f}")
    
    # Compute and print averages
    avg_mse_pred = np.mean(mse_summary["mse_pred"])
    avg_mse_adjusted = np.mean(mse_summary["mse_adjusted"])
    print(f"\nAverage MSE (Pred): {avg_mse_pred:.4f}")
    print(f"Average MSE (Adjusted): {avg_mse_adjusted:.4f}")



def evaluate_observation(config, device):
    """
    Load models, dataset, and evaluate the depth model with visualization.
    
    Parameters:
    - config: Configuration dictionary.
    - device: Device to run the model (CPU or GPU).
    """
    dataset_dir = config["dataset_dir"]
    log_dir_depth = config["log_dir_depth"]
    split_file = config["split_file"]

    # Load dataset
    with open(split_file, "r") as f:
        split = AttrDict(yaml.safe_load(f))

    test_set = GridSeqDataset(
        dataset_dir,
        split.val[:config["num_of_scenes"]],
        L=config["L"],
    )

    # Load depth network
    depth_net = depth_net_pl.load_from_checkpoint(
        checkpoint_path=log_dir_depth,
        d_min=config["d_min"],
        d_max=config["d_max"],
        d_hyp=config["d_hyp"],
        D=config["D"],
    ).to(device)

    # Evaluate depth model with visualization
    evaluate_depth_model(depth_net, test_set, device, fov=config["fov"])


def main():
    """
    Main function to set up configurations and run evaluation.
    """
    # Hardcoded configuration
    config = {
        "dataset_dir": "/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full",
        "log_dir_depth": "/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/modules/Final_wights/depth/final_depth_model_checkpoint.ckpt",
        "split_file": "/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full/split.yaml",
        "L": 0,
        "D": 128,
        "d_min": 0.1,
        "d_max": 15.0,
        "d_hyp": -0.2,
        "fov": 80,  # Field of view in degrees
        "num_of_scenes": 100,
    }

    # Set device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #print(f"Using device: {device}")

    # Run evaluation
    evaluate_observation(config, device)


if __name__ == "__main__":
    main()


/home/mascher/anaconda3/envs/yuval_fp/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mascher/anaconda3/envs/yuval_


Average MSE (Pred): 0.8853
Average MSE (Adjusted): 0.8880
